In [272]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [273]:
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print('Proteins shape:',proteins.shape)
proteins.head()

Proteins shape: (232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [274]:
import warnings
import numpy as np
import pandas as pd

### Machine Learning ###
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Optimization ###
import optuna

### Viz ###
from colorama import Fore

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [275]:
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
m_ = Fore.MAGENTA
r_ = Fore.RED
y_ = Fore.YELLOW

In [276]:
def smape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    return 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

In [277]:
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print('Proteins shape:',proteins.shape)
proteins.head()

Proteins shape: (232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [278]:
proteins_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
print('Proteins test shape:',proteins_test.shape)
proteins_test.head()

Proteins test shape: (453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [279]:
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
print('Peptides shape:', peptides.shape)
peptides.head()

Peptides shape: (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [280]:
peptides_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')
print('Peptides test shape:', peptides_test.shape)
peptides_test.head()

Peptides test shape: (2057, 7)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.3,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.6,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.4,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.0,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.2,0


In [281]:
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
print('Clinical shape:', clinical.shape)
clinical.head()

Clinical shape: (2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [282]:
test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')
print('Test shape:', test.shape)
test.head()

Test shape: (16, 6)


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


In [283]:
proteins.groupby('visit_id').agg({'UniProt':'nunique','patient_id':'count','NPX':['min','max','mean','std']}).reset_index()

visit_id UniProt patient_id       NPX                             \
               nunique      count       min          max          mean   
0      10053_0     165        165  2497.840  269126000.0  2.856580e+06   
1     10053_12     171        171  5800.870  270030000.0  2.728871e+06   
2     10053_18     208        208  1334.110  278835000.0  2.509967e+06   
3     10138_12     217        217  2520.240  365582000.0  3.002583e+06   
4     10138_24     219        219  1436.940  396894000.0  3.068891e+06   
...        ...     ...        ...       ...          ...           ...   
1108   8699_24     216        216   756.551  346067000.0  3.064059e+06   
1109    942_12     212        212  1722.770  330558000.0  2.613298e+06   
1110    942_24     217        217  1339.150  336769000.0  2.616142e+06   
1111    942_48     216        216  1272.480  358059000.0  2.768442e+06   
1112     942_6     216        216  2491.690  290111000.0  2.476880e+06   

                    
               std  
0     2.131630e+07  
1     2.092162e+07  
2     1.969453e+07  
3     2.516170e+07  
4     2.716806e+07  
...            ...  
1108  2.409420e+07  
1109  2.295228e+07  
1110  2.312662e+07  
1111  2.460543e+07  
1112  2.002174e+07  

[1113 rows x 7 columns]

In [284]:
proteins2=proteins.drop(columns=['patient_id','UniProt'])
peptides2=peptides.drop(columns=['patient_id','UniProt','Peptide'])
p1=peptides2.groupby('visit_id').agg({'PeptideAbundance': ['min','max','mean']}).reset_index()
pr2=proteins2.groupby('visit_id').agg({'NPX':['min','max','mean']}).reset_index()
p1.columns=['visit_id','pa_min','pa_max','pa_mean']
pr2.columns=['visit_id','NPX_min','NPX_max','NPX_mean']

In [285]:
pr2.columns=['visit_id','NPX_min','NPX_max','NPX_mean']

In [286]:
pr2

,visit_id,NPX_min,NPX_max,NPX_mean
0,10053_0,2497.840,269126000.0,2.856580e+06
1,10053_12,5800.870,270030000.0,2.728871e+06
2,10053_18,1334.110,278835000.0,2.509967e+06
3,10138_12,2520.240,365582000.0,3.002583e+06
4,10138_24,1436.940,396894000.0,3.068891e+06
...,...,...,...,...
1108,8699_24,756.551,346067000.0,3.064059e+06
1109,942_12,1722.770,330558000.0,2.613298e+06
1110,942_24,1339.150,336769000.0,2.616142e+06
1111,942_48,1272.480,358059000.0,2.768442e+06


In [287]:
df_0_1 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_1']]
df_0_2 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_2']]
df_0_3 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_3']]
df_0_4 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_4']]

In [288]:
s_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [289]:
proteins[['visit_month']].max()

visit_month    108
dtype: int64

In [290]:
peptides[['visit_month']].max()

visit_month    108
dtype: int64

In [291]:
proteins.isnull().sum()

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64

In [292]:
peptides.isnull().sum()

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64

In [293]:
clinical.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [294]:
clinical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2615 non-null   object 
 1   patient_id                           2615 non-null   int64  
 2   visit_month                          2615 non-null   int64  
 3   updrs_1                              2614 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2590 non-null   float64
 6   updrs_4                              1577 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 163.6+ KB


In [295]:
test

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


In [296]:
peptides

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [297]:
proteins

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [298]:
clinical.value_counts()

visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  upd23b_clinical_state_on_medication
10138_0   10138       0            8.0      6.0      15.0     0.0      On                                     1
51846_18  51846       18           8.0      4.0      9.0      0.0      On                                     1
51879_0   51879       0            2.0      8.0      47.0     0.0      Off                                    1
51846_6   51846       6            8.0      2.0      8.0      0.0      On                                     1
51846_48  51846       48           5.0      1.0      12.0     2.0      Off                                    1
                                                                                                             ..
27893_30  27893       30           11.0     15.0     27.0     0.0      On                                     1
27893_24  27893       24           7.0      15.0     27.0     0.0      On                                    

In [299]:
c2= pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [300]:
proteins[['patient_id']].value_counts()

patient_id
62723         1929
20404         1924
23391         1919
26210         1912
18183         1899
              ... 
55240          429
42086          427
25911          419
52119          417
41930          199
Length: 248, dtype: int64

In [301]:
peptides[['patient_id']].value_counts()

patient_id
20404         8272
23391         8147
62723         8111
26210         8076
18183         7965
              ... 
55240         1810
52119         1804
25911         1782
42086         1781
41930          876
Length: 248, dtype: int64

In [302]:
proteins[['visit_month']].value_counts()

visit_month
0              40587
12             37467
24             36225
36             31156
48             28838
6              25991
60             17572
84              6288
72              4407
18              1657
108              855
3                641
96               423
54               418
30               216
dtype: int64

In [303]:
clinical[['patient_id']].value_counts()

patient_id
55096         17
57108         17
15009         17
16778         16
52266         16
              ..
13968          5
20792          5
62732          4
60443          3
14450          3
Length: 248, dtype: int64

In [304]:
c2[['patient_id']].value_counts()

patient_id
12956         6
58377         6
8614          6
57367         6
9251          6
             ..
53143         1
53241         1
40776         1
12732         1
34311         1
Length: 771, dtype: int64

In [306]:
c2.columns=clinical.columns

In [307]:
all_clinical=pd.concat([clinical,c2])

In [308]:
all_clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
...,...,...,...,...,...,...,...,...
2218,65382_0,65382,0,NaN,NaN,0.0,NaN,NaN
2219,65405_0,65405,0,5.0,16.0,31.0,0.0,NaN
2220,65405_5,65405,5,NaN,NaN,57.0,NaN,NaN
2221,65530_0,65530,0,10.0,6.0,24.0,0.0,NaN


In [309]:
all_clinical2=all_clinical
all_clinical2['upd23b_clinical_state_on_medication']=all_clinical[['upd23b_clinical_state_on_medication']].fillna('Off')
all_clinical2['updrs_4']=all_clinical[['updrs_4']].fillna(0)

In [310]:
all_clinical3=pd.get_dummies(all_clinical2,columns=['upd23b_clinical_state_on_medication'])
all_clinical3

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication_Off,upd23b_clinical_state_on_medication_On
0,55_0,55,0,10.0,6.0,15.0,0.0,1,0
1,55_3,55,3,10.0,7.0,25.0,0.0,1,0
2,55_6,55,6,8.0,10.0,34.0,0.0,1,0
3,55_9,55,9,8.0,9.0,30.0,0.0,0,1
4,55_12,55,12,10.0,10.0,41.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...
2218,65382_0,65382,0,NaN,NaN,0.0,0.0,1,0
2219,65405_0,65405,0,5.0,16.0,31.0,0.0,1,0
2220,65405_5,65405,5,NaN,NaN,57.0,0.0,1,0
2221,65530_0,65530,0,10.0,6.0,24.0,0.0,1,0


In [311]:
peptides

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [312]:
peptides[peptides['Peptide']=='GLVSWGNIPC(UniMod_4)GSK']

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
899,55_0,0,55,Q92876,GLVSWGNIPC(UniMod_4)GSK,724168.0
1802,1517_0,0,1517,Q92876,GLVSWGNIPC(UniMod_4)GSK,606324.0
2680,1923_0,0,1923,Q92876,GLVSWGNIPC(UniMod_4)GSK,1161040.0
3587,2660_0,0,2660,Q92876,GLVSWGNIPC(UniMod_4)GSK,808147.0
4467,3636_0,0,3636,Q92876,GLVSWGNIPC(UniMod_4)GSK,330256.0
...,...,...,...,...,...,...
978172,58648_96,96,58648,Q92876,GLVSWGNIPC(UniMod_4)GSK,464939.0
979063,5645_108,108,5645,Q92876,GLVSWGNIPC(UniMod_4)GSK,777961.0
979973,12703_108,108,12703,Q92876,GLVSWGNIPC(UniMod_4)GSK,755370.0
980895,55096_108,108,55096,Q92876,GLVSWGNIPC(UniMod_4)GSK,415757.0


In [313]:
proteins[proteins['UniProt']=='P07333']

,visit_id,visit_month,patient_id,UniProt,NPX
96,55_0,0,55,P07333,54219.0
315,1517_0,0,1517,P07333,31639.9
524,1923_0,0,1923,P07333,43414.7
735,2660_0,0,2660,P07333,74919.3
945,3636_0,0,3636,P07333,45944.4
...,...,...,...,...,...
231557,5645_96,96,5645,P07333,25904.0
231766,58648_96,96,58648,P07333,33764.0
231979,5645_108,108,5645,P07333,20807.3
232193,12703_108,108,12703,P07333,46080.7


In [314]:
train_all=pd.merge(p1,all_clinical3,how='right')
train_all2=pd.merge(pr2,train_all,how='right')
train_all2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4838 entries, 0 to 4837
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   visit_id                                 4838 non-null   object 
 1   NPX_min                                  1068 non-null   float64
 2   NPX_max                                  1068 non-null   float64
 3   NPX_mean                                 1068 non-null   float64
 4   pa_min                                   1068 non-null   float64
 5   pa_max                                   1068 non-null   float64
 6   pa_mean                                  1068 non-null   float64
 7   patient_id                               4838 non-null   int64  
 8   visit_month                              4838 non-null   int64  
 9   updrs_1                                  4624 non-null   float64
 10  updrs_2                                  4622 no

In [315]:
train_all2

,visit_id,NPX_min,NPX_max,NPX_mean,pa_min,pa_max,pa_mean,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication_Off,upd23b_clinical_state_on_medication_On
0,55_0,3417.71,390239000.0,3.180508e+06,2148.35,99701200.0,748153.907014,55,0,10.0,6.0,15.0,0.0,1,0
1,55_3,NaN,NaN,NaN,NaN,NaN,NaN,55,3,10.0,7.0,25.0,0.0,1,0
2,55_6,2505.10,347572000.0,2.942039e+06,1444.29,77597700.0,685218.599872,55,6,8.0,10.0,34.0,0.0,1,0
3,55_9,NaN,NaN,NaN,NaN,NaN,NaN,55,9,8.0,9.0,30.0,0.0,0,1
4,55_12,3592.52,383507000.0,3.145608e+06,2814.33,86513500.0,740535.423563,55,12,10.0,10.0,41.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,65382_0,NaN,NaN,NaN,NaN,NaN,NaN,65382,0,NaN,NaN,0.0,0.0,1,0
4834,65405_0,NaN,NaN,NaN,NaN,NaN,NaN,65405,0,5.0,16.0,31.0,0.0,1,0
4835,65405_5,NaN,NaN,NaN,NaN,NaN,NaN,65405,5,NaN,NaN,57.0,0.0,1,0
4836,65530_0,NaN,NaN,NaN,NaN,NaN,NaN,65530,0,10.0,6.0,24.0,0.0,1,0


In [316]:
train_all3=train_all2.dropna()

In [317]:
train_all3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 2610
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   visit_id                                 1058 non-null   object 
 1   NPX_min                                  1058 non-null   float64
 2   NPX_max                                  1058 non-null   float64
 3   NPX_mean                                 1058 non-null   float64
 4   pa_min                                   1058 non-null   float64
 5   pa_max                                   1058 non-null   float64
 6   pa_mean                                  1058 non-null   float64
 7   patient_id                               1058 non-null   int64  
 8   visit_month                              1058 non-null   int64  
 9   updrs_1                                  1058 non-null   float64
 10  updrs_2                                  1058 no

In [318]:
featuresall=train_all3.columns.difference(['visit_id','patient_id','upd23b_clinical_state_on_medication_Off','upd23b_clinical_state_on_medication_On','updrs_1','updrs_2','updrs_3','updrs_4'])
features0=train_all3.columns.difference(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4'])
features1=train_all3.columns.difference(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4'])
features2=train_all3.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
features3=train_all3.columns.difference(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4'])
features_1=train_all3.columns.difference(['visit_id','patient_id','updrs_4'])


In [319]:
features2

Index(['NPX_max', 'NPX_mean', 'NPX_min', 'pa_max', 'pa_mean', 'pa_min',
       'upd23b_clinical_state_on_medication_Off',
       'upd23b_clinical_state_on_medication_On', 'visit_month'],
      dtype='object')

In [320]:
# Robust scaling 수행
#scaler = RobustScaler()
#train_all32 = scaler.fit_transform(train_all3[features2.tolist()])
#train_all322 = pd.DataFrame(train_all32,columns=features2)
#train_all33 = scaler.fit_transform(train_all3[features3.tolist()])
#train_all333 = pd.DataFrame(train_all33,columns=features3)
#train_all3_1 = scaler.fit_transform(train_all3[features_1.tolist()])
#train_all3_11 = pd.DataFrame(train_all3_1,columns=features_1)
#train_all34 = scaler.fit_transform(train_all3[features4.tolist()])
#train_all344 = pd.DataFrame(train_all34,columns=features4)

In [321]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
train_all30 = scaler.fit_transform(train_all3[featuresall.tolist()])
train_all300 = pd.DataFrame(train_all30,columns=featuresall)


In [322]:
train_all300[['updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication_Off','upd23b_clinical_state_on_medication_On']]=train_all3[['updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication_Off','upd23b_clinical_state_on_medication_On']].reset_index(drop=True)

In [323]:
train_all300

,NPX_max,NPX_mean,NPX_min,pa_max,pa_mean,pa_min,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication_Off,upd23b_clinical_state_on_medication_On
0,1.222037,1.021539,1.491930,1.715358,0.933677,50.147054,-0.571429,10.0,6.0,15.0,0.0,1,0
1,0.610438,0.532852,0.651682,0.430429,0.414391,32.826973,-0.428571,8.0,10.0,34.0,0.0,1,0
2,1.125539,0.950019,1.652879,0.948726,0.870816,66.530356,-0.285714,10.0,10.0,41.0,0.0,0,1
3,2.076975,1.501736,1.958582,1.469196,1.503596,72.333319,0.285714,17.0,18.0,51.0,0.0,0,1
4,-0.213222,-0.420379,0.639335,-0.681363,-0.430712,1.205242,-0.428571,8.0,2.0,21.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,-0.263951,-0.743202,0.145449,0.029229,-0.708632,-0.482308,0.571429,11.0,17.0,46.0,1.0,0,1
1054,-1.069951,-1.246876,-1.420141,-0.943354,-1.273452,-0.255609,1.428571,11.0,15.0,45.0,4.0,1,0
1055,-0.243869,0.873318,1.364108,-0.375919,0.665888,0.515179,-0.571429,2.0,6.0,16.0,0.0,1,0
1056,0.205023,0.672433,1.353778,0.021788,0.532953,0.387282,-0.285714,4.0,7.0,14.0,0.0,1,0


In [324]:
train_all3

,visit_id,NPX_min,NPX_max,NPX_mean,pa_min,pa_max,pa_mean,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication_Off,upd23b_clinical_state_on_medication_On
0,55_0,3417.710,390239000.0,3.180508e+06,2148.3500,99701200.0,748153.907014,55,0,10.0,6.0,15.0,0.0,1,0
2,55_6,2505.100,347572000.0,2.942039e+06,1444.2900,77597700.0,685218.599872,55,6,8.0,10.0,34.0,0.0,1,0
4,55_12,3592.520,383507000.0,3.145608e+06,2814.3300,86513500.0,740535.423563,55,12,10.0,10.0,41.0,0.0,0,1
8,55_36,3924.550,449882000.0,3.414835e+06,3050.2200,95466700.0,817225.824754,55,36,17.0,18.0,51.0,0.0,0,1
15,942_6,2491.690,290111000.0,2.476880e+06,158.8690,58472500.0,582795.558517,942,6,8.0,2.0,21.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,64674_48,1955.270,286572000.0,2.319349e+06,90.2702,70696200.0,549112.789519,64674,48,11.0,17.0,46.0,1.0,0,1
2598,64674_84,254.851,230343000.0,2.073565e+06,99.4855,53965700.0,480658.964631,64674,84,11.0,15.0,45.0,4.0,1,0
2600,65043_0,3278.880,287973000.0,3.108179e+06,130.8180,63726800.0,715699.056675,65043,0,2.0,6.0,16.0,0.0,1,0
2604,65043_12,3267.660,319289000.0,3.010151e+06,125.6190,70568200.0,699587.777590,65043,12,4.0,7.0,14.0,0.0,1,0


In [325]:
proteins_test[['patient_id']].value_counts()

patient_id
50423         227
3342          226
dtype: int64

In [326]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=train_all300[features0]
y=train_all300.updrs_2
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [XGBRegressor(), CatBoostRegressor(), LGBMRegressor(), RandomForestRegressor()]
params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_2 = {}
oof_preds2 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_
    models_2[i] = best_model2
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model2.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model2.predict(val_X)
    oof_preds2[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Best SMAPE: 98.60004653223538
0:	learn: 6.1175693	total: 605us	remaining: 59.9ms
1:	learn: 6.1117904	total: 1.04ms	remaining: 51.1ms
2:	learn: 6.1070396	total: 1.38ms	remaining: 44.7ms
3:	learn: 6.1016964	total: 1.76ms	remaining: 42.2ms
4:	learn: 6.0957470	total: 2.13ms	remaining: 40.5ms
5:	learn: 6.0906346	total: 2.53ms	remaining: 39.7ms
6:	learn: 6.0854354	total: 2.92ms	remaining: 38.7ms
7:	learn: 6.0807786	total: 3.31ms	remaining: 38.1ms
8:	learn: 6.0763861	total: 3.73ms	remaining: 37.8ms
9:	learn: 6.0710743	total: 4.12ms	remaining: 37.1ms
10:	learn: 6.0660122	total: 4.52ms	remaining: 36.6ms
11:	learn: 6.0614095	total: 4.92ms	remaining: 36.1ms
12:	learn: 6.0566931	total: 5.3ms	remaining: 35.5ms
13:	learn: 6.0525769	total: 5.69ms	remaining: 34.9ms
14:	learn: 6.0475722	total: 6.09ms	remaining: 34.5ms
15:	learn: 6.0434189	total: 6.51ms	remaining: 34.2ms
16:	learn: 6.0394884	total: 6.91ms	remaining: 33.7ms
17:

In [330]:
models_2

{0: XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, ...),
 1: <catboost.core.CatBoostRegressor at 0x7f360d933ad0>,
 2: LGBMRegressor(learning_rate=0.01, max_depth=3),
 3: RandomForestRegressor(max_depth=5, min_samples_split=4, n_estimators=300)}

In [ ]:
#ensemble_preds2 = np.mean(dfe, axis=1)
#train_all300['updrs_2'] = ensemble_preds2
#train_all300

from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

# 각 모델의 가중치 초기화
w0 = np.ones(len(dfe)) / len(dfe)

# 가중치 최적화 함수 정의
def objective(weights):
    # 예측값 가중합 계산
    y_pred = np.sum(weights.reshape(-1,1) * dfe, axis=0)
    # SMAPE 계산
    score = smape(y_train, y_pred)
    # 손실값 반환
    return score

# 최적화 알고리즘 적용
result = minimize(objective, w0, method='Nelder-Mead')

# 최적 가중치 출력
weights = result.x
print(weights)

# 가중합 예측값 계산
y_pred2 = np.sum(weights.reshape(-1,1) * dfe, axis=0)


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=train_all300[features0]
y=train_all300.updrs_3
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [XGBRegressor(), CatBoostRegressor(), LGBMRegressor(), RandomForestRegressor()]
params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_3 = {}
oof_preds3 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model3 = grid_search.best_estimator_
    models_3[i] = best_model3
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model3.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model3.predict(val_X)
    oof_preds3[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

In [ ]:
dfe3 = pd.DataFrame(oof_preds3)
dfe3

ensemble_preds3 = np.mean(dfe3, axis=1)
ensemble_preds3

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=train_all300[features0]
y=train_all300.updrs_1
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [XGBRegressor(), CatBoostRegressor(), LGBMRegressor(), RandomForestRegressor()]
params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_1 = {}
oof_preds1 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model1 = grid_search.best_estimator_
    models_1[i] = best_model1
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model1.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model1.predict(val_X)
    oof_preds1[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

In [ ]:
dfe1 = pd.DataFrame(oof_preds1)
dfe1

ensemble_preds1 = np.mean(dfe1, axis=1)
ensemble_preds1

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=train_all300[features0]
y=train_all300.updrs_4
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [XGBRegressor(), CatBoostRegressor(), LGBMRegressor(), RandomForestRegressor()]
params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_4 = {}
oof_preds4 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model4 = grid_search.best_estimator_
    models_4[i] = best_model4
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model4.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model4.predict(val_X)
    oof_preds4[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

In [ ]:
dfe4 = pd.DataFrame(oof_preds4)
dfe4

ensemble_preds4 = np.mean(dfe4, axis=1)
ensemble_preds4

In [331]:
peptides_test

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.30,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.60,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.40,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.00,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.20,0
...,...,...,...,...,...,...,...
2052,3342_6,6,3342,Q9Y646,AIINLAVYGK,6142.76,6
2053,3342_6,6,3342,Q9Y646,LALLVDTVGPR,23602.30,6
2054,3342_6,6,3342,Q9Y646,VGALASLIR,21728.20,6
2055,3342_6,6,3342,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,4253.17,6


In [332]:
proteins_t2=proteins_test.drop(columns=['patient_id','UniProt'])
peptides_t2=peptides_test.drop(columns=['patient_id','UniProt','Peptide'])
p_t1=peptides_t2.groupby('visit_id').agg({'PeptideAbundance': ['min','max','mean']}).reset_index()
pr_t2=proteins_t2.groupby('visit_id').agg({'NPX':['min','max','mean']}).reset_index()
p_t1.columns=['visit_id','pa_min','pa_max','pa_mean']
pr_t2.columns=['visit_id','NPX_min','NPX_max','NPX_mean']

In [333]:
proteins_t2[['visit_id']].value_counts()

visit_id
50423_0     227
3342_6      226
dtype: int64

In [334]:
peptides_test[['visit_id']].value_counts()

visit_id
3342_6      1036
50423_0     1021
dtype: int64

In [335]:
p_t1

,visit_id,pa_min,pa_max,pa_mean
0,3342_6,1391.470,34270800.0,581048.801390
1,50423_0,886.086,55622600.0,725015.018899


In [336]:
pr_t2

,visit_id,NPX_min,NPX_max,NPX_mean
0,3342_6,2853.12,295860000.0,2.663569e+06
1,50423_0,2287.04,439208000.0,3.260972e+06


In [351]:
best_model2.predict(testall)

ValueError: X has 8 features, but RandomForestRegressor is expecting 9 features as input.

In [348]:
testall=pd.merge(p_t1,pr_t2,on='visit_id')

In [349]:
testall['month']=[6,0]

In [350]:
testall

,visit_id,pa_min,pa_max,pa_mean,NPX_min,NPX_max,NPX_mean,month
0,3342_6,1391.470,34270800.0,581048.801390,2853.12,295860000.0,2.663569e+06,6
1,50423_0,886.086,55622600.0,725015.018899,2287.04,439208000.0,3.260972e+06,0


In [342]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()


In [ ]:
def map_test(x):
    updrs = x.split('_')[2] + '_' + x.split('_')[3]
    month = int(x.split('_plus_')[1].split('_')[0])
    visit_id = x.split('_')[0] + '_' + x.split('_')[1]
    # set all predictions 0 where updrs equals 'updrs_4'
    if updrs=='updrs_3':
#         rating = updrs_3_pred[month]
        rating = df[df.visit_id == visit_id]['pred2'].values[0]
    elif updrs=='updrs_4':
        rating = 0
    elif updrs =='updrs_1':
        rating = df[df.visit_id == visit_id]['pred0'].values[0]
    else:
        rating = df[df.visit_id == visit_id]['pred1'].values[0]
    return rating

counter = 0
# The API will deliver four dataframes in this specific order:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    df = test[['visit_id']].drop_duplicates('visit_id')
    pred_0 = features(df[['visit_id']], test_proteins, test_peptides, 0)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']